In [166]:
import os
from dotenv import load_dotenv
from IPython.display import display
from openai import OpenAI
import json
from scraping import fetch_category_news, fetch_article_body, custom_css
from prompts import SYSTEM_PROMPT, get_user_prompt, final_prompt
from dotenv import load_dotenv
import gradio as gr


In [168]:
load_dotenv(override=True)
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"


google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [169]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

In [ ]:
#For running locally
OLLAMA_BASE_URL = "http://localhost:11434/v1"
MODEL = 'llama3.2'
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [ ]:

SOURCES = {
    "Israel News": [
       "https://www.jpost.com/Rss/RssFeedsFrontPage.aspx",
        "https://www.timesofisrael.com/feed/"
    ],
    "Technology": [
        "https://www.geektime.co.il/feed/",
        "https://techcrunch.com/feed/"
        "https://www.theverge.com/rss/index.xml"
    ],
    "Business & Finance": [
        "http://feeds.bbci.co.uk/news/business/rss.xml",
        "https://feeds.a.dj.com/rss/WSJcomUSBusiness.xml"
    ],
    "Science & Health": [
        "https://www.sciencenews.org/feed",
        "https://www.nature.com/nature.rss"
    ]
}

In [ ]:
# selected_category = "Israel News"
# links = fetch_category_news(selected_category, SOURCES)

# links

In [172]:
def AI_select_top_10(selected_category,links):
    """
    Uses the AI model to analyze a list of raw news links and select the top relevant stories.
    Returns a JSON-formatted list of URLs that strictly match the chosen category and quality criteria.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": get_user_prompt(selected_category, links)}
    ]
    response = gemini.chat.completions.create(
        model = MODEL,
        messages = messages,
        response_format={ "type": "json_object" }
    )
    content = response.choices[0].message.content
    data = json.loads(content)
    return data.get("urls", [])

def getDailyBriefly(selected_category):
    """
    Core logic handler: Coordinates news fetching and AI selection, then 
    constructs a string to format the final briefing report.
    """
    links = fetch_category_news(selected_category, SOURCES)
    selected_urls = AI_select_top_10(selected_category,links)

    all_content_for_final_report = ""

    for url in selected_urls:
        article_text, image_url = fetch_article_body(url) 
        if "Error fetching" in article_text:
            print(f"Skipping {url} due to fetch error")
            continue

        title_line = article_text.split("\n")[0] if article_text else "No Title"
        
        source = url.split("/")[2]
        
        all_content_for_final_report += f"""
            ARTICLE_START
            TITLE: {title_line}
            IMAGE_URL: {image_url if image_url else 'None'}
            URL: {url}
            CONTENT: {article_text}
            ARTICLE_END
            """  

    final_messages = final_prompt(all_content_for_final_report)

    final_briefing = gemini.chat.completions.create(
    model=MODEL,
    messages=final_messages
    ).choices[0].message.content

    final_briefing_clean = "\n".join([line.strip() for line in final_briefing.splitlines()])

    return final_briefing_clean

In [ ]:
"""
UI Orchestrator: Triggers the briefing logic and handles the visibility 
    toggle between the input settings and the rendered result briefing.
"""
with gr.Blocks(css=custom_css, title="DAILY BRIEFING") as demo:
    gr.Markdown("# 📰 DAILY BRIEFING")
    
    with gr.Group() as input_area:
        category = gr.Dropdown(list(SOURCES.keys()), label="Select Category")
        btn = gr.Button("Flash Briefing", variant="primary")
    
    output_area = gr.Markdown(visible=False)

    def run_and_switch(category):
        result = getDailyBriefly(category)
        return gr.update(visible=False), gr.update(value=result, visible=True)

    btn.click(run_and_switch, inputs=[category], outputs=[input_area, output_area])

demo.launch()

C:\Users\Alster\AppData\Local\Temp\ipykernel_22604\3994182570.py:5: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=custom_css, title="DAILY BRIEFING") as demo:


* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


Scanning: https://www.jpost.com/Rss/RssFeedsFrontPage.aspx
Scanning: https://www.timesofisrael.com/feed/
Total articles found: 30
